In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pickle
import cv2
from keras.models import load_model
import numpy as np
import tensorflow as tf
from skimage import io
import pandas as pd
import glob
import os

In [ ]:
label_dict = {
    0: "TUMOR",
    1: "STROMA",
    2: "COMPLEX",
    3: "LYMPHO",
    4: "DEBRIS",
    5: "MUCOSA",
    6: "ADIPOSE",
    7: "EMPTY"
}

In [ ]:
cnn_model = load_model('/content/drive/MyDrive/Filters_colorectal/best_model.h5')

In [ ]:
features_layer1 = tf.keras.models.Model(
    inputs=cnn_model.inputs,
    outputs=cnn_model.get_layer(name="layer1024").output,
)

In [ ]:
def feat(img):
  img = cv2.resize(img, (150, 150))
  img = cv2.bilateralFilter(img,3,90,90)
  img = np.array(img)
  img = np.expand_dims(img, axis=0)
  feat = features_layer1(img)
  feat = np.array(feat)
  return feat

In [ ]:
def cnn_pred(img):
  img = cv2.resize(img, (150, 150))
  img = cv2.bilateralFilter(img,3,90,90)
  img = np.array(img)
  img = np.expand_dims(img, axis=0)
  pred_cnn = cnn_model.predict(img)
  pred = np.argmax(pred_cnn)
  print("The predicted clas is", label_dict[pred])

In [ ]:
img = io.imread('/content/drive/MyDrive/png/07_ADIPOSE/image_101.png',0)
cnn_pred(img)

1/1 [==============================] - 0s 497ms/step
The predicted clas is ADIPOSE


#feature selection

In [ ]:
features = feat(img).flatten()

data = []
header = [f"f_{i}" for i in range(1, 1025)]
data.append(features)
df = pd.DataFrame(data, columns=header)

In [ ]:
features.shape

(1024,)

In [ ]:
def give_selected(df):
  with open('/content/drive/MyDrive/Filters_colorectal/RFE/rfe.pkl', 'rb') as model_file:
    rfe_model = pickle.load(model_file)
  s_f = rfe_model.support_
  selected_features = []

  for feature, selected in zip(df.columns, s_f):
    if selected:
        selected_features.append(feature)
  df = df[selected_features].copy()
  return df

In [ ]:
sub_df = give_selected(df)

In [ ]:
sub_df

,f_6,f_12,f_13,f_19,f_20,f_25,f_26,f_31,f_40,f_45,...,f_993,f_994,f_996,f_1002,f_1005,f_1008,f_1011,f_1015,f_1017,f_1024
0,4.596785,0.0,43.142281,0.0,0.0,0.599575,0.0,10.986145,0.0,0.0,...,0.0,7.763802,14.238513,0.0,19.560871,4.491216,0.0,0.432424,10.692594,0.0


#stacking


In [ ]:
def get_stacking_pred(final_df):
  with open('/content/drive/MyDrive/Filters_colorectal/RFE/knn.pkl', 'rb') as model_file:
    knn_model = pickle.load(model_file)
  with open('/content/drive/MyDrive/Filters_colorectal/RFE/svm.pkl', 'rb') as model_file:
    svm_model = pickle.load(model_file)
  with open('/content/drive/MyDrive/Filters_colorectal/RFE/rfc.pkl', 'rb') as model_file:
    rfc_model = pickle.load(model_file)
  with open('/content/drive/MyDrive/Filters_colorectal/RFE/mlp.pkl', 'rb') as model_file:
    mlp_model = pickle.load(model_file)
  with open('/content/drive/MyDrive/Filters_colorectal/RFE/fs_best_40.pkl', 'rb') as model_file:
    stack_model = pickle.load(model_file)

  probs_model1_test = knn_model.predict_proba(final_df)
  probs_model2_test = svm_model.predict_proba(final_df)
  probs_model3_test = rfc_model.predict_proba(final_df)
  probs_model4_test = mlp_model.predict_proba(final_df)

  meta_features_test = np.concatenate((probs_model1_test, probs_model2_test, probs_model3_test, probs_model4_test), axis=1)

  pred = stack_model.predict(meta_features_test)

  pred_p = stack_model.predict_proba(meta_features_test)
  pred_p = max(pred_p.flatten())
  print(pred_p*100)
  print("The predicted clas is", label_dict[int(pred[0])])

In [ ]:
get_stacking_pred(sub_df)

46.97066463030997
The predicted clas is ADIPOSE
